In [1]:
from util import PATH_FOR_OUR_TRAINING_DATA, UPPER_LIMIT_OF_IMAGES, TARGET_SHAPE
import pandas as pd

In [2]:
import tensorflow
from DS_Generator import DS_Generator
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [3]:
ds = DS_Generator().generate()

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=TARGET_SHAPE+(3,)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [7]:
model.fit(
    ds,
    epochs=5)

Epoch 1/5
799/799 [==============================] - 106s 119ms/step - loss: nan
Epoch 2/5
799/799 [==============================] - 94s 118ms/step - loss: nan
Epoch 3/5
799/799 [==============================] - 92s 114ms/step - loss: nan
Epoch 4/5
799/799 [==============================] - 95s 118ms/step - loss: nan
Epoch 5/5
799/799 [==============================] - 95s 118ms/step - loss: nan
